## Download Example Data

In [1]:
# ! FILEID="1OO0tUguZMyQ1d37K7F9jiwV7mm_z2yuD" && FILENAME="example_data.npy" && wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILEID" -O $FILENAME && rm -rf /tmp/cookies.txt

## Import Numba CUDA

In [2]:
from numba import cuda
import numpy as np
import math

In [3]:
data = np.load('example_data.npy')

## Original (CPU-based)

In [4]:
def ridge_detection(f, thres):
    count = np.zeros(f.shape)
    for i in range(len(f)):
        for j in range(len(f[i])):
            if (
                i > 0
                and j > 0
                and i < (len(f) - 1)
                and j < (len(f[i]) - 1)
                and f[i, j] > thres
                and ~np.isnan(f[i, j])
            ):
                step_i = i
                step_j = j
                for k in range(1000):
                    if (
                        step_i == 0
                        or step_j == 0
                        or step_i == (len(f) - 1)
                        or step_j == (len(f[i]) - 1)
                    ):
                        break
                    index = np.nanargmax(
                        f[step_i - 1 : step_i + 2, step_j - 1 : step_j + 2].data
                    )
                    vmax = np.nanmax(
                        f[step_i - 1 : step_i + 2, step_j - 1 : step_j + 2].data
                    )
                    if index == 4 or vmax == f[step_i, step_j] or np.isnan(vmax):
                        break
                    row = int(index / 3)
                    col = index % 3
                    count[step_i - 1 + row, step_j - 1 + col] += 1
                    step_i = step_i - 1 + row
                    step_j = step_j - 1 + col
    return count

In [5]:
%%time
results = ridge_detection(data, 0)

CPU times: user 6min 14s, sys: 15.9 s, total: 6min 30s
Wall time: 6min 6s


In [6]:
# %timeit -r 7 -n 1 ridge_detection(data, 0)